# Визуализация данных с использованием Plotly

Plotly – это библиотека с открытым исходным кодом для Python и R, которая отлично подходит для создания красивых и интерактивных визуализаций.

Plotly — библиотека для визуализации данных, состоящая из нескольких частей:
- Front-End на JS
- Back-End на Python (за основу взята библиотека Seaborn)
- Back-End на R

Plotly позволяет строить интерактивные визуализации. 

Документация:

- [Plotly](https://plotly.com/)

- [Plotly.Express](https://plotly.com/python/plotly-express/)

- [Plotly.Dash](https://dash.plotly.com/) - построение дашбордов



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv('datasets/aggrigation_logs_per_week.csv')
df

In [ ]:
df.info()

## Задание 1: Проведите предобработку набора данных

Преобразуйте столбцы, содержащие вещественные значения к типу float.

In [ ]:
# ваш код

In [ ]:
df['s_all_avg'] = df['s_all_avg'].str.replace(',','.')
df['s_all_avg']= df['s_all_avg'].astype('float')
df ['s_course_viewed_avg'] =df ['s_course_viewed_avg'].str.replace(',','.').astype('float')
df['s_q_attempt_viewed_avg'] = df['s_q_attempt_viewed_avg'].str.replace(',','.').astype('float')

In [ ]:
df['s_a_submission_status_viewed_avg'] = df['s_a_submission_status_viewed_avg'].str.replace(',','.').astype('float')

## Примеры визуализаций
Построим столбчатую диаграмму количества студентов по каждому семестру. 
- Метод px.bar() - передаем данные для построения диаграммы
- Функция fig.update_layout() - настройки общего оформления графика (layout): заголовки осей, легенда, фон, отступы, шрифты и другие параметры визуализации.

In [ ]:
# Группируем по семестру и считаем количество уникальных студентов
df_grouped = df.groupby("Num_Sem")["userid"].nunique().reset_index()
df_grouped.rename(columns={"userid": "num_students"}, inplace=True)

# Строим столбчатую диаграмму
fig = px.bar(
    df_grouped, 
    x="Num_Sem", 
    y="num_students", 
    text="num_students",  # Подписи с количеством студентов
    color="num_students", 
    title="Количество студентов, изучающих дисциплины в семестр",
    labels={"Num_Sem": "Семестр", "num_students": "Количество студентов"},
    color_continuous_scale="viridis"
)

fig.update_layout(
    xaxis_title="Номер семестра",
    yaxis_title="Количество студентов",
    template="plotly_white"
)

fig.show()

Построим точечную диаграмму количества студентов по каждому семестру. 

px.scatter() - передаем данные для построения диаграммы. 

In [ ]:
# Группируем по семестру и считаем количество уникальных студентов
df_grouped = df.groupby("Num_Sem")["userid"].nunique().reset_index()
df_grouped.rename(columns={"userid": "num_students"}, inplace=True)

# Строим точечную диаграмму (scatter plot)
fig = px.scatter(
    df_grouped, 
    x="Num_Sem", 
    y="num_students", 
    text="num_students",  # Подписи значений
    color="num_students", 
    size="num_students",  # Размер точек зависит от количества студентов
    title="Количество студентов, изучающих дисциплины в семестр",
    labels={"Num_Sem": "Семестр", "num_students": "Количество студентов"},
    color_continuous_scale="viridis"
)

fig.update_layout(
    xaxis_title="Номер семестра",
    yaxis_title="Количество студентов",
    template="plotly_white"
)

# Добавляем подписи точек выше самих точек
fig.update_traces(
    textposition='top center',
    marker=dict(
        line=dict(width=1, color='DarkSlateGrey')
    )
)

fig.show()

Построим интерактивный график активности студентов по неделям
Используем plotly.express.area для построения графика изменения активности студентов (s_all) по неделям (num_week). Дополнительно, разделим данные по курсам (Kurs), чтобы увидеть, как активность различается среди студентов разных курсов.
Построить график px.area, где:
- Ось X — num_week (номер недели семестра)
- Ось Y — s_all (общее количество событий)
- Цветовая группировка — Kurs (номер курса студента)
- Добавить анимацию по семестрам (Num_Sem), чтобы увидеть динамику активности студентов разных курсов (свойство -  animation_frame).

In [ ]:
# Преобразуем колонки с числами, если они хранятся как строки
numeric_cols = ["s_all", "s_all_avg", "s_course_viewed", "s_course_viewed_avg",
                "s_q_attempt_viewed", "s_q_attempt_viewed_avg", "s_a_course_module_viewed",
                "s_a_course_module_viewed_avg", "s_a_submission_status_viewed", "s_a_submission_status_viewed_avg"]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")  # Преобразуем к float

# Строим график активности студентов по неделям
fig = px.area(
    df, 
    x="num_week", 
    y="s_all", 
    color="Kurs",  # Разделение по курсам
    line_group="Kurs", 
    title="Динамика активности студентов в течение семестра",
    labels={"num_week": "Неделя", "s_all": "Количество событий", "Kurs": "Курс"},
    animation_frame="Num_Sem"  # Анимация по семестрам
)

fig.show()

## Задание 2. Влияние формы обучения на активность
Решите гипотезу: студенты, обучающиеся на контракте, менее активны на портале по сравнению с бюджетниками.

Что визуализировать: 
Различие в среднем числе взаимодействий между группами ("Name_OsnO") по следующим показателями: "s_all_avg", "s_course_viewed_avg", "s_q_attempt_viewed_avg".
Необходимо использовать в  plotly.express Boxplot распределения активности для бюджетников и контрактников.

In [ ]:
fig1 = px.box(
    df,
    x="Name_OsnO",
    y="s_all_avg",
    color="Name_OsnO",
    title="Общее количество взаимодействий (s_all_avg)",
    labels={"Name_OsnO": "Форма обучения", "s_all_avg": "Среднее количество взаимодействий"}
)
fig1.show()

In [ ]:
fig2 = px.box(
    df,
    x="Name_OsnO",
    y="s_course_viewed_avg",
    color="Name_OsnO",
    title="Среднее количество просмотра курсов (s_course_viewed_avg)",
    labels={"Name_OsnO": "Форма обучения", "s_course_viewed_avg": "Среднее количество просмотра курсов"}
)
fig2.show()

In [ ]:
fig3 = px.box(
    df,
    x="Name_OsnO",
    y="s_q_attempt_viewed_avg",
    color="Name_OsnO",
    title="Среднее количество просмотров попыток тестов (s_q_attempt_viewed_avg)",
    labels={"Name_OsnO": "Форма обучения", "s_q_attempt_viewed_avg": "Среднее количество просмотров попыток тестов"}
)
fig3.show()

Ваш ответ: ....

# Задание 3. Динамика активности студентов по неделям
Гипотеза: Активность студентов увеличивается перед сессией и уменьшается после неё.

Постройте линейный график изменения активности (s_all) по неделям (num_week).
Сравнить активность по курсам (kurs). Все курсы должны отображаться на одной диаграмме. 

In [ ]:
df_grouped = df.groupby(['num_week', 'Kurs'])['s_all'].mean().reset_index()
fig = px.line(
    df_grouped,
    x='num_week',
    y='s_all',
    color='Kurs',
    markers=True,
    title='Активность студентов по неделям (s_all)',
    labels={
        'num_week': 'Номер недели',
        's_all': 'Средняя активность за неделю (s_all)',
        'Kurs': 'Курс'
    }
)

fig.show()

Ваш ответ: ....

## Задание 4: Проанализировать динамику активности студентов с разным уровнем успеваемости (NameR_Level) в течение всего семестра.
Решите гипотезу: успеваемость студентов зависит от динамики их активности на образовательном портале в течение всего семестра. 

Построить линейный график (px.line), где:
X-ось — num_week (номер недели семестра)
Y-ось — s_all (общее количество событий)
Цветовая группировка — NameR_Level (успеваемость: отличники, хорошисты, троечники, двоечники)

In [ ]:
df_grouped = df.groupby(['num_week', 'NameR_Level'])['s_all'].mean().reset_index()

fig = px.line(
    df_grouped,
    x='num_week',
    y='s_all',
    color='NameR_Level',
    markers=True,
    title='Активность студентов по неделям (s_all)',
    labels={
        'num_week': 'Номер недели',
        's_all': 'Средняя активность за неделю (s_all)',
        'NameR_Level': 'Оценка'
    }
)

fig.show()

Ваш ответ: ....

# Задание 5. Различие в активности бакалавров и магистров
Гипотеза: Магистранты взаимодействуют с курсами иначе, чем бакалавры.

Построить воронку (метод go.Funnel()) активности студентов (s_course_viewed, s_q_attempt_viewed, s_a_submission_status_viewed) в зависимости от уровня образования (leveled).

In [ ]:
# Проверяем, какие значения есть в LevelEd
print(df["LevelEd"].unique())  

# закодируем типы обучения
df["LevelEd"] = df["LevelEd"].replace({1: "бакалавриат", 2: "магистратура", 3: "специалитет", 4: "аспирантура"})

# Группируем данные
df_level = df.groupby("LevelEd")[["s_course_viewed", "s_q_attempt_viewed", "s_a_submission_status_viewed"]].mean().reset_index()

print(df_level)

# Строим воронку
fig = go.Figure()
fig.add_trace(go.Funnel(
    name="Бакалавриат",
    y=["Просмотры курса", "Просмотры тестов", "Отправка заданий"],
    x=df_level[df_level["LevelEd"] == "бакалавриат"].iloc[:, 1:].values.flatten()
))
fig.add_trace(go.Funnel(
    name="Магистратура",
    y=["Просмотры курса", "Просмотры тестов", "Отправка заданий"],
    x=df_level[df_level["LevelEd"] == "магистратура"].iloc[:, 1:].values.flatten()
))

fig.update_layout(title="Различие в активности бакалавров и магистров")

# Показываем график
fig.show()

Ваш ответ: ....

## Задание 6. Различие в активности отличников, хорошистов, троечников и двоечников 
Гипотеза: Студенты с разным уровнем успеваемостью взаимодействуют с курсами иначе.

Построить воронку активности студентов (s_course_viewed, s_q_attempt_viewed, s_a_submission_status_viewed) в зависимости от уровня успеваемости (NameR_Level).

In [ ]:
df_level = df.groupby("NameR_Level")[["s_course_viewed", "s_q_attempt_viewed", "s_a_submission_status_viewed"]].mean().reset_index()

print(df_level)

# Строим воронку
fig = go.Figure()
fig.add_trace(go.Funnel(
    name="Двоечники",
    y=["Просмотры курса", "Просмотры тестов", "Отправка заданий"],
    x=df_level[df_level["NameR_Level"] == 2].iloc[:, 1:].values.flatten()
))

fig.add_trace(go.Funnel(
    name="Троечники",
    y=["Просмотры курса", "Просмотры тестов", "Отправка заданий"],
    x=df_level[df_level["NameR_Level"] == 3].iloc[:, 1:].values.flatten()
))

fig.add_trace(go.Funnel(
    name="Хорошисты",
    y=["Просмотры курса", "Просмотры тестов", "Отправка заданий"],
    x=df_level[df_level["NameR_Level"] == 4].iloc[:, 1:].values.flatten()
))

fig.add_trace(go.Funnel(
    name="Отличники",
    y=["Просмотры курса", "Просмотры тестов", "Отправка заданий"],
    x=df_level[df_level["NameR_Level"] == 5].iloc[:, 1:].values.flatten()
))

fig.update_layout(title="Различие в активности студентов с разной успеваемостью")

# Показываем график
fig.show()

## Задание 7:  Распределение студентов по кафедрам 
Определим распределение студентов по кафедрам, однаковое ли количество студентов?

Постройте горизонтальную столбчатую диаграмму (bar chart) для сравнения количества студентов на разных кафедрах.
Добавим интерактивность, цветовую палитру и подписи. На диаграмме должны быть выведены названия кафедр (возьмите данную информацию из Task 4.)

In [ ]:
department_names = {
    1: 'АиИИ', 2: 'АСУ', 3: 'АЭПиМ', 4: 'БИиИТ', 5: 'ВИ',
    6: 'ВТиП', 7: 'ГМДиОПИ', 8: 'ГМиТТК', 9: 'ГМУиУП', 10: 'Дизайна',
    11: 'ДиСО', 12: 'ИиИБ', 13: 'ИТМ', 14: 'ЛиП', 15: 'ЛиУТС',
    16: 'ЛПиМ', 17: 'Менеджм.', 18: 'МиТОДиМ', 19: 'МиХТ', 20: 'ПиСЗ',
    21: 'ПиЭММО', 22: 'ПМиИ', 23: 'ПОиД', 24: 'Психол.', 25: 'ПЭиБЖД',
    26: 'РМПИ', 27: 'РЯОЯиМК', 28: 'СРиППО', 29: 'CC', 30: 'ТиЭС',
    31: 'ТОМ', 32: 'ТССА', 33: 'УиИС', 34: 'УСиБА', 35: 'Физики',
    36: 'Физкульт.', 37: 'Химии', 38: 'ХОМ', 39: 'ЦДОМ', 40: 'ЭиМЭ',
    41: 'Эконом.', 42: 'ЭПП', 43: 'ЯиЛ'
}

df_unique = df.drop_duplicates(subset='userid')
student_counts = df_unique['Depart'].value_counts().reset_index()
student_counts.columns = ['Depart', 'Количество студентов']

In [ ]:
student_counts['Кафедра'] = student_counts['Depart'].map(department_names)

In [ ]:
student_counts

In [ ]:
fig = px.bar(
    student_counts,
    x='Количество студентов',
    y='Кафедра',
    orientation='h',
    color='Кафедра',
    title='Количество уникальных студентов по кафедрам'
)

fig.show()

## Задание 8: Зависимость среднего балла от количества студентов на кафедре
Гипотеза: количество студентов, закрепленных на отдельной кафедре влияет на средний балл успеваемости по кафедрам.

Построить диаграмму рассевания по количеству студентов, среднему баллу, наванию кафедры. Параметр  size='num_students' - количество студентов. 

In [ ]:
df.info()

In [ ]:
df[:5]

In [ ]:
df_depart = (df.groupby('Depart').agg(avg_mark=('NameR_Level', 'mean'), count_students=('userid', 'nunique')).reset_index())
df_depart['Depart'] = df_depart['Depart'].map(department_names)
df_depart

In [ ]:
fig = px.scatter(
    df_depart,
    x='count_students',
    y='avg_mark',
    size='count_students',
    color='Depart',
    hover_name='Depart',
    title='Зависимость среднего балла от количества студентов на кафедре',
    labels={'count_students': 'Число студентов', 'avg_mark': 'Средний балл'}
)

fig.show()

Ваш ответ: ....

## Задание 9: Результаты кластеризации студентов (часть 1)
Примечание: В Task5 вы реализовали кластеризацию студентов деятельности студентов, разбив их на 3 кластера. 
Вам необходимо визуализировать результаты кластеризации.

Гипотеза: студентов с низкой активностью на портале меньше всего.

Постройте круговую диаграмму с количеством студентов (не логов) по каждому кластеру (названия кластеров должны иметь категориальные значения).

In [ ]:
data = {
    'cluster': [1, 2, 0],
    'count': [318, 1181, 4817]
}
data = pd.DataFrame(data)

In [ ]:
cluster_names = {
    0: "Низкая активность",
    1: "Высокая активность",
    2: "Средняя активность"
}
data["cluster_name"] = data["cluster"].map(cluster_names)

In [ ]:
fig = px.pie(
    data,
    names='cluster_name',
    values='count',
    title='Распределение студентов по активности',
    color='cluster'
)
fig.show()

## Задание 10: Результаты кластеризации студентов (часть 2) 

Примечание: В Task5 вы реализовали кластеризацию студентов деятельности студентов, разбив их на 3 кластера. 
Вам необходимо визуализировать результаты кластеризации.

Гипотеза: кластеры студентов с низкой и высокой активностью не пересекаются при группировке студентов по результатам экзаменов и со срезом студентов, которые получили 2 и 5. 

Постройте диаграмму рассеяния, где отображаются кластеры активности студентов, которые получили только 2 и 5. 


In [ ]:
filtered_data = df[['userid', 'NameR_Level']].copy()
filtered_data = filtered_data[filtered_data['NameR_Level'].isin([2, 5])]
print(filtered_data['NameR_Level'].unique())
print(filtered_data.head())

In [ ]:
df_cluster = pd.read_csv('clusters_result.csv')
df_cluster.head()

In [ ]:
df_merged = pd.merge(filtered_data, df_cluster, on='userid')
df_merged.head()

In [ ]:
cluster_map = {
    0: "Низкая активность",
    1: "Высокая активность",
    2: "Средняя активность"
}
df_merged['cluster'] = df_merged['cluster'].map(cluster_map)

In [ ]:
fig = px.scatter(
    df_merged,
    x="avg_s_all",
    y="NameR_Level",
    color="cluster",
    title="Диаграмма рассеяния: Активность vs Экзаменационная оценка (только 2 и 5)",
    labels={"avg_s_all": "Средняя активность", "exam_score": "Оценка"},
    color_discrete_map={
        "Низкая активность": "red",
        "Средняя активность": "orange",
        "Высокая активность": "green"
    },
    hover_data=["userid"]
)

fig.show()